# G03: Weighted Phase Lag Index (wPLI)

**Duration**: 50 minutes  
**Prerequisites**: G02 (Phase Lag Index)  
**Next**: H01 (Envelope Correlation)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Explain PLI's noise sensitivity problem
2. Define wPLI as a weighted improvement over PLI
3. Implement wPLI computation
4. Interpret wPLI values
5. Compare PLI, wPLI, and PLV across scenarios
6. Apply wPLI to hyperscanning analysis
7. Choose appropriately among phase-based metrics

In [ ]:
# Required imports
import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt
from scipy import signal
from typing import Any

# Visualization settings
plt.style.use('seaborn-v0_8-whitegrid')

# Color palette
COLORS = {
    'signal_1': '#2E86AB',
    'signal_2': '#E94F37',
    'accent': '#A23B72',
    'highlight': '#F18F01',
    'warning': '#C73E1D',
    'subject_1': '#2E86AB',
    'subject_2': '#E94F37',
}

# Random seed for reproducibility
np.random.seed(42)

---

## Section 1: The Problem with PLI

In the previous notebook (G02), we learned that **Phase Lag Index (PLI)** offers a crucial advantage over PLV: it's robust to volume conduction because it ignores zero-phase-lag contributions. PLI only considers the *sign* of the phase difference, not its magnitude.

However, this strength comes with a significant **weakness**: the sign function is discontinuous at zero. When phase differences fluctuate around a small positive value, even tiny noise perturbations can push some samples across zero, causing **sign flips**. This happens because:

- A phase difference of +0.01 radians → sign = +1
- A phase difference of -0.01 radians → sign = -1

These sign flips add noise to the PLI estimate. The problem is **particularly severe** when the true phase difference is small—exactly the situation where we might have genuine connectivity with a small lag. Noise makes the signs appear balanced, artificially reducing PLI toward zero.

**What we need**: A way to weight contributions by how far they are from zero, so that uncertain phase differences near zero contribute less to our estimate.

In [ ]:
# Visualization 1: PLI noise sensitivity illustration

np.random.seed(42)
n_points = 200

# True phase difference: small positive value (0.15 radians ≈ 8.6°)
true_lag = 0.15
noise_std = 0.2

# Phase differences with noise
phase_diffs = true_lag + np.random.normal(0, noise_std, n_points)

# Compute signs
signs = np.sign(np.sin(phase_diffs))

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Panel 1: Phase differences over time
ax1 = axes[0, 0]
time = np.arange(n_points)
colors = [COLORS['signal_1'] if s > 0 else COLORS['warning'] for s in signs]
ax1.scatter(time, phase_diffs, c=colors, s=20, alpha=0.7)
ax1.axhline(0, color='black', linestyle='-', linewidth=2, label='Zero line')
ax1.axhline(true_lag, color=COLORS['highlight'], linestyle='--', linewidth=2, label=f'True lag = {true_lag:.2f} rad')
ax1.set_xlabel('Sample', fontsize=11)
ax1.set_ylabel('Phase difference (rad)', fontsize=11)
ax1.set_title('Phase differences fluctuating around small positive value', fontsize=12)
ax1.legend(loc='upper right')
ax1.set_ylim(-0.8, 0.8)

# Panel 2: Sign series
ax2 = axes[0, 1]
ax2.step(time, signs, where='mid', color=COLORS['signal_1'], linewidth=1.5)
for i, s in enumerate(signs):
    if i > 0 and signs[i] != signs[i-1]:
        ax2.axvline(i, color=COLORS['warning'], alpha=0.3, linewidth=1)
ax2.axhline(0, color='black', linestyle='-', linewidth=1)
ax2.set_xlabel('Sample', fontsize=11)
ax2.set_ylabel('Sign', fontsize=11)
ax2.set_title('Sign flips caused by noise crossing zero', fontsize=12)
ax2.set_ylim(-1.5, 1.5)
ax2.set_yticks([-1, 0, 1])

# Panel 3: Distribution of phase differences
ax3 = axes[1, 0]
ax3.hist(phase_diffs, bins=30, color=COLORS['signal_1'], alpha=0.7, edgecolor='white')
ax3.axvline(0, color='black', linestyle='-', linewidth=2)
ax3.axvline(true_lag, color=COLORS['highlight'], linestyle='--', linewidth=2, label=f'True lag')
ax3.axvline(np.mean(phase_diffs), color=COLORS['accent'], linestyle=':', linewidth=2, label=f'Mean = {np.mean(phase_diffs):.3f}')
ax3.set_xlabel('Phase difference (rad)', fontsize=11)
ax3.set_ylabel('Count', fontsize=11)
ax3.set_title('Distribution crosses zero due to noise', fontsize=12)
ax3.legend()

# Panel 4: PLI calculation breakdown
ax4 = axes[1, 1]
n_positive = np.sum(signs > 0)
n_negative = np.sum(signs < 0)
pli_value = np.abs(np.mean(signs))

bars = ax4.bar(['Positive (+1)', 'Negative (-1)'], [n_positive, n_negative], 
               color=[COLORS['signal_1'], COLORS['warning']], edgecolor='white', linewidth=2)
ax4.set_ylabel('Count', fontsize=11)
ax4.set_title(f'Sign balance → PLI = {pli_value:.3f}\n(Should be higher for true lag!)', fontsize=12)

# Add count labels on bars
for bar, count in zip(bars, [n_positive, n_negative]):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
             str(count), ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.suptitle('PLI Noise Sensitivity: Small phase differences are unstable', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"True phase lag: {true_lag:.3f} rad ({np.degrees(true_lag):.1f}°)")
print(f"Positive signs: {n_positive}, Negative signs: {n_negative}")
print(f"PLI = |mean(signs)| = |({n_positive} - {n_negative})/{n_points}| = {pli_value:.3f}")
print(f"\n⚠️ PLI is artificially LOW because noise causes sign flips!")

---

## Section 2: The wPLI Solution — Weighting by Magnitude

The **Weighted Phase Lag Index (wPLI)** addresses PLI's noise sensitivity with a clever insight: **don't give equal weight to all phase differences**.

The key ideas are:

1. **Phase differences near zero** are more likely to be:
   - Volume conduction artifacts (which we want to ignore)
   - Noise fluctuations (which destabilize our estimate)
   - Therefore, give them **LESS weight**

2. **Phase differences far from zero** are more likely to be:
   - Real neural connectivity
   - Therefore, give them **MORE weight**

**How do we implement this weighting?** We use **|sin(Δφ)|** as the weight:

- When Δφ = 0 or ±π → sin(Δφ) = 0 → weight = 0 (no contribution)
- When Δφ = ±π/2 → |sin(Δφ)| = 1 → maximum weight

This elegantly down-weights contributions from phase differences near zero (where sign flips are likely) while preserving the volume conduction robustness.

In [ ]:
# Visualization 2: Weighting function

phase_range = np.linspace(-np.pi, np.pi, 500)
weight = np.abs(np.sin(phase_range))

fig, ax = plt.subplots(figsize=(10, 5))

ax.fill_between(phase_range, weight, alpha=0.3, color=COLORS['signal_1'])
ax.plot(phase_range, weight, color=COLORS['signal_1'], linewidth=3, label='Weight = |sin(Δφ)|')

# Mark key points
key_points = [(-np.pi, 0), (-np.pi/2, 1), (0, 0), (np.pi/2, 1), (np.pi, 0)]
labels = ['-π', '-π/2', '0', 'π/2', 'π']
for (x, y), label in zip(key_points, labels):
    ax.plot(x, y, 'o', markersize=12, color=COLORS['highlight'], zorder=5)
    offset = 0.12 if y == 0 else -0.15
    ax.annotate(f'({label}, {y})', (x, y + offset), ha='center', fontsize=10, fontweight='bold')

# Highlight zero-weight regions
ax.axhspan(0, 0.1, alpha=0.2, color=COLORS['warning'], label='Low weight zone (near 0, ±π)')

ax.set_xlabel('Phase difference Δφ (rad)', fontsize=12)
ax.set_ylabel('Weight', fontsize=12)
ax.set_title('wPLI Weighting Function: Zero weight at 0 and ±π', fontsize=14, fontweight='bold')
ax.set_xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
ax.set_xticklabels(['-π', '-π/2', '0', 'π/2', 'π'])
ax.set_xlim(-np.pi - 0.3, np.pi + 0.3)
ax.set_ylim(-0.1, 1.2)
ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
# Visualization 3: PLI vs wPLI weighting comparison

phase_range = np.linspace(-np.pi, np.pi, 500)

# PLI contribution: just the sign (constant magnitude)
pli_contribution = np.sign(np.sin(phase_range))

# wPLI contribution: sign weighted by magnitude
wpli_contribution = np.sin(phase_range)  # sin already includes sign and magnitude

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# PLI weighting
ax1 = axes[0]
colors_pli = [COLORS['signal_1'] if s > 0 else COLORS['signal_2'] for s in pli_contribution]
ax1.fill_between(phase_range, 0, pli_contribution, where=(pli_contribution > 0), 
                  color=COLORS['signal_1'], alpha=0.5, label='Positive (+1)')
ax1.fill_between(phase_range, 0, pli_contribution, where=(pli_contribution < 0), 
                  color=COLORS['signal_2'], alpha=0.5, label='Negative (-1)')
ax1.axhline(0, color='black', linewidth=1)
ax1.axvline(0, color='black', linewidth=1, linestyle='--', alpha=0.5)
ax1.set_xlabel('Phase difference Δφ (rad)', fontsize=12)
ax1.set_ylabel('Contribution to PLI', fontsize=12)
ax1.set_title('PLI: All non-zero phases contribute equally\n(sign only, magnitude ignored)', fontsize=12, fontweight='bold')
ax1.set_xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
ax1.set_xticklabels(['-π', '-π/2', '0', 'π/2', 'π'])
ax1.set_ylim(-1.5, 1.5)
ax1.legend(loc='upper right')

# wPLI weighting
ax2 = axes[1]
ax2.fill_between(phase_range, 0, wpli_contribution, where=(wpli_contribution > 0), 
                  color=COLORS['signal_1'], alpha=0.5, label='Positive')
ax2.fill_between(phase_range, 0, wpli_contribution, where=(wpli_contribution < 0), 
                  color=COLORS['signal_2'], alpha=0.5, label='Negative')
ax2.axhline(0, color='black', linewidth=1)
ax2.axvline(0, color='black', linewidth=1, linestyle='--', alpha=0.5)
ax2.set_xlabel('Phase difference Δφ (rad)', fontsize=12)
ax2.set_ylabel('Contribution to wPLI', fontsize=12)
ax2.set_title('wPLI: Phases near zero contribute LESS\n(weighted by |sin(Δφ)|)', fontsize=12, fontweight='bold')
ax2.set_xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
ax2.set_xticklabels(['-π', '-π/2', '0', 'π/2', 'π'])
ax2.set_ylim(-1.5, 1.5)
ax2.legend(loc='upper right')

plt.suptitle('PLI vs wPLI Weighting Scheme', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---

## Section 3: The wPLI Formula

The **Weighted Phase Lag Index** was introduced by Vinck et al. (2011). The formula can be expressed in terms of the cross-spectrum or phase differences:

### Cross-spectrum formulation:
$$wPLI = \frac{|\langle |\text{Im}(S_{xy})| \cdot \text{sign}(\text{Im}(S_{xy})) \rangle|}{\langle |\text{Im}(S_{xy})| \rangle}$$

### Phase difference formulation:
$$wPLI = \frac{|\sum_t |\sin(\Delta\phi_t)| \cdot \text{sign}(\sin(\Delta\phi_t))|}{\sum_t |\sin(\Delta\phi_t)|}$$

### Simplified form:
$$wPLI = \frac{|\sum_t \sin(\Delta\phi_t)|}{\sum_t |\sin(\Delta\phi_t)|}$$

**Breaking it down:**

- **Numerator**: Sum of sin(Δφ) values (signed) → can cancel if balanced
- **Denominator**: Sum of |sin(Δφ)| values (always positive) → normalization
- **Result**: Ratio measures the asymmetry, weighted by magnitude

### Properties:

- **Range**: 0 to 1
- **wPLI = 0**: Symmetric phase distribution OR phases concentrated near zero
- **wPLI = 1**: All phases at exactly ±π/2 with consistent sign
- **Key advantage**: Less sensitive to noise near zero than PLI

In [ ]:
# Visualization 4: Formula breakdown

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Example phase differences
np.random.seed(123)
example_phases = np.array([0.8, -0.2, 0.9, 0.1, -0.1, 0.7, -0.3, 0.6, 0.4, -0.15])

# Panel 1: sin(Δφ) values
ax1 = axes[0]
sin_values = np.sin(example_phases)
colors = [COLORS['signal_1'] if v > 0 else COLORS['signal_2'] for v in sin_values]
bars1 = ax1.bar(range(len(sin_values)), sin_values, color=colors, edgecolor='white', linewidth=1)
ax1.axhline(0, color='black', linewidth=1)
ax1.set_xlabel('Sample', fontsize=11)
ax1.set_ylabel('sin(Δφ)', fontsize=11)
ax1.set_title('Step 1: Compute sin(Δφ)\n(signed values)', fontsize=12, fontweight='bold')
ax1.set_ylim(-1, 1)

# Panel 2: Numerator and Denominator
ax2 = axes[1]
numerator = np.abs(np.sum(sin_values))
denominator = np.sum(np.abs(sin_values))
bars2 = ax2.bar(['|Σ sin(Δφ)|\n(Numerator)', 'Σ |sin(Δφ)|\n(Denominator)'], 
                [numerator, denominator],
                color=[COLORS['signal_1'], COLORS['accent']], edgecolor='white', linewidth=2)
for bar, val in zip(bars2, [numerator, denominator]):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
             f'{val:.3f}', ha='center', fontsize=12, fontweight='bold')
ax2.set_ylabel('Value', fontsize=11)
ax2.set_title('Step 2: Sum signed vs unsigned', fontsize=12, fontweight='bold')
ax2.set_ylim(0, 6)

# Panel 3: Final wPLI
ax3 = axes[2]
wpli_val = numerator / denominator if denominator > 0 else 0
ax3.bar(['wPLI'], [wpli_val], color=COLORS['highlight'], edgecolor='white', linewidth=2, width=0.5)
ax3.text(0, wpli_val + 0.05, f'{wpli_val:.3f}', ha='center', fontsize=14, fontweight='bold')
ax3.set_ylabel('wPLI', fontsize=11)
ax3.set_title('Step 3: Ratio\nwPLI = Num / Denom', fontsize=12, fontweight='bold')
ax3.set_ylim(0, 1.2)
ax3.axhline(1.0, color='gray', linestyle='--', alpha=0.5, label='Maximum = 1')
ax3.legend()

plt.suptitle('wPLI Formula Components', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"sin(Δφ) values: {sin_values.round(3)}")
print(f"Numerator: |Σ sin(Δφ)| = |{np.sum(sin_values):.3f}| = {numerator:.3f}")
print(f"Denominator: Σ |sin(Δφ)| = {denominator:.3f}")
print(f"wPLI = {numerator:.3f} / {denominator:.3f} = {wpli_val:.3f}")

---

## Section 4: Implementing wPLI

Now let's implement the wPLI computation. The pipeline is:

1. **Band-pass filter** both signals
2. **Extract phases** using Hilbert transform
3. **Compute phase differences** Δφ(t)
4. **Compute sin(Δφ(t))** for each time point
5. **Numerator**: |sum of sin values|
6. **Denominator**: sum of |sin values|
7. **wPLI** = numerator / denominator

In [ ]:
def bandpass_filter(
    data: NDArray[np.float64],
    fs: float,
    band: tuple[float, float],
    order: int = 4
) -> NDArray[np.float64]:
    """
    Apply bandpass filter to signal.
    
    Parameters
    ----------
    data : NDArray[np.float64]
        Input signal (1D or 2D with shape n_channels x n_samples)
    fs : float
        Sampling frequency in Hz
    band : tuple[float, float]
        Frequency band (low, high) in Hz
    order : int, optional
        Filter order (default: 4)
        
    Returns
    -------
    NDArray[np.float64]
        Filtered signal
    """
    nyq = fs / 2
    low, high = band[0] / nyq, band[1] / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    return signal.filtfilt(b, a, data, axis=-1)


def extract_phase(
    data: NDArray[np.float64]
) -> NDArray[np.float64]:
    """
    Extract instantaneous phase using Hilbert transform.
    
    Parameters
    ----------
    data : NDArray[np.float64]
        Input signal (1D or 2D)
        
    Returns
    -------
    NDArray[np.float64]
        Instantaneous phase in radians [-π, π]
    """
    analytic = signal.hilbert(data, axis=-1)
    return np.angle(analytic)

In [ ]:
def compute_wpli_from_phases(
    phase_x: NDArray[np.float64],
    phase_y: NDArray[np.float64]
) -> float:
    """
    Compute Weighted Phase Lag Index from pre-computed phases.
    
    Parameters
    ----------
    phase_x : NDArray[np.float64]
        Instantaneous phase of signal x (1D array)
    phase_y : NDArray[np.float64]
        Instantaneous phase of signal y (1D array)
        
    Returns
    -------
    float
        wPLI value in range [0, 1]
        
    Notes
    -----
    wPLI = |Σ sin(Δφ)| / Σ |sin(Δφ)|
    
    This formulation weights phase differences by their distance from
    zero, making wPLI more robust to noise than PLI.
    """
    # Compute phase differences
    phase_diff = phase_x - phase_y
    
    # Compute sin of phase differences
    sin_phase_diff = np.sin(phase_diff)
    
    # Numerator: absolute value of sum (signed)
    numerator = np.abs(np.sum(sin_phase_diff))
    
    # Denominator: sum of absolute values (unsigned)
    denominator = np.sum(np.abs(sin_phase_diff))
    
    # Handle division by zero
    if denominator == 0:
        return 0.0
    
    wpli = numerator / denominator
    
    # Clip to [0, 1] for numerical stability
    return float(np.clip(wpli, 0.0, 1.0))


def compute_wpli(
    x: NDArray[np.float64],
    y: NDArray[np.float64],
    fs: float,
    band: tuple[float, float],
    filter_order: int = 4
) -> float:
    """
    Compute Weighted Phase Lag Index between two signals.
    
    Parameters
    ----------
    x : NDArray[np.float64]
        First signal (1D array)
    y : NDArray[np.float64]
        Second signal (1D array)
    fs : float
        Sampling frequency in Hz
    band : tuple[float, float]
        Frequency band (low, high) in Hz
    filter_order : int, optional
        Butterworth filter order (default: 4)
        
    Returns
    -------
    float
        wPLI value in range [0, 1]
    """
    # Bandpass filter
    x_filt = bandpass_filter(x, fs, band, filter_order)
    y_filt = bandpass_filter(y, fs, band, filter_order)
    
    # Extract phases
    phase_x = extract_phase(x_filt)
    phase_y = extract_phase(y_filt)
    
    return compute_wpli_from_phases(phase_x, phase_y)


def compute_wpli_from_cross_spectrum(
    cross_spectrum: NDArray[np.complex128]
) -> float:
    """
    Compute wPLI from cross-spectral density.
    
    Parameters
    ----------
    cross_spectrum : NDArray[np.complex128]
        Complex cross-spectral density values
        
    Returns
    -------
    float
        wPLI value in range [0, 1]
        
    Notes
    -----
    Uses the imaginary part of the cross-spectrum:
    wPLI = |mean(|Im(Sxy)| * sign(Im(Sxy)))| / mean(|Im(Sxy)|)
    """
    # Get imaginary part
    imag = np.imag(cross_spectrum)
    
    # Numerator: |sum of signed imaginary parts weighted by magnitude|
    numerator = np.abs(np.sum(np.abs(imag) * np.sign(imag)))
    
    # Denominator: sum of |imaginary parts|
    denominator = np.sum(np.abs(imag))
    
    if denominator == 0:
        return 0.0
    
    return float(np.clip(numerator / denominator, 0.0, 1.0))

In [ ]:
# Visualization 5: wPLI computation pipeline

fig, axes = plt.subplots(2, 3, figsize=(14, 8))

# Generate example signals
np.random.seed(42)
fs = 500.0
duration = 2.0
t = np.arange(0, duration, 1/fs)
n_samples = len(t)
freq = 10.0
phase_lag = np.pi / 3  # 60 degrees

# Create phase-locked signals
x = np.sin(2 * np.pi * freq * t) + 0.3 * np.random.randn(n_samples)
y = np.sin(2 * np.pi * freq * t + phase_lag) + 0.3 * np.random.randn(n_samples)

# Step 1: Raw signals
ax1 = axes[0, 0]
ax1.plot(t[:200], x[:200], color=COLORS['signal_1'], linewidth=1.5, label='Signal X')
ax1.plot(t[:200], y[:200], color=COLORS['signal_2'], linewidth=1.5, label='Signal Y')
ax1.set_xlabel('Time (s)', fontsize=10)
ax1.set_ylabel('Amplitude', fontsize=10)
ax1.set_title('Step 1: Raw signals', fontsize=11, fontweight='bold')
ax1.legend(loc='upper right', fontsize=9)

# Step 2: Bandpass filter
ax2 = axes[0, 1]
x_filt = bandpass_filter(x, fs, (8, 12))
y_filt = bandpass_filter(y, fs, (8, 12))
ax2.plot(t[:200], x_filt[:200], color=COLORS['signal_1'], linewidth=1.5, label='X filtered')
ax2.plot(t[:200], y_filt[:200], color=COLORS['signal_2'], linewidth=1.5, label='Y filtered')
ax2.set_xlabel('Time (s)', fontsize=10)
ax2.set_ylabel('Amplitude', fontsize=10)
ax2.set_title('Step 2: Bandpass filter (8-12 Hz)', fontsize=11, fontweight='bold')
ax2.legend(loc='upper right', fontsize=9)

# Step 3: Extract phases
ax3 = axes[0, 2]
phase_x = extract_phase(x_filt)
phase_y = extract_phase(y_filt)
ax3.plot(t[:200], phase_x[:200], color=COLORS['signal_1'], linewidth=1.5, label='Phase X')
ax3.plot(t[:200], phase_y[:200], color=COLORS['signal_2'], linewidth=1.5, label='Phase Y')
ax3.set_xlabel('Time (s)', fontsize=10)
ax3.set_ylabel('Phase (rad)', fontsize=10)
ax3.set_title('Step 3: Extract phases (Hilbert)', fontsize=11, fontweight='bold')
ax3.legend(loc='upper right', fontsize=9)

# Step 4: Phase differences
ax4 = axes[1, 0]
phase_diff = phase_x - phase_y
ax4.plot(t[:200], phase_diff[:200], color=COLORS['accent'], linewidth=1.5)
ax4.axhline(0, color='black', linestyle='--', alpha=0.5)
ax4.axhline(phase_lag, color=COLORS['highlight'], linestyle='--', label=f'True lag = {phase_lag:.2f}')
ax4.set_xlabel('Time (s)', fontsize=10)
ax4.set_ylabel('Δφ (rad)', fontsize=10)
ax4.set_title('Step 4: Phase differences', fontsize=11, fontweight='bold')
ax4.legend(loc='upper right', fontsize=9)

# Step 5: sin(Δφ)
ax5 = axes[1, 1]
sin_diff = np.sin(phase_diff)
ax5.plot(t[:200], sin_diff[:200], color=COLORS['signal_1'], linewidth=1.5)
ax5.axhline(0, color='black', linestyle='--', alpha=0.5)
ax5.fill_between(t[:200], 0, sin_diff[:200], where=(sin_diff[:200] > 0), 
                  color=COLORS['signal_1'], alpha=0.3)
ax5.fill_between(t[:200], 0, sin_diff[:200], where=(sin_diff[:200] < 0), 
                  color=COLORS['signal_2'], alpha=0.3)
ax5.set_xlabel('Time (s)', fontsize=10)
ax5.set_ylabel('sin(Δφ)', fontsize=10)
ax5.set_title('Step 5: Compute sin(Δφ)', fontsize=11, fontweight='bold')

# Step 6: Final wPLI
ax6 = axes[1, 2]
numerator = np.abs(np.sum(sin_diff))
denominator = np.sum(np.abs(sin_diff))
wpli_val = numerator / denominator

bars = ax6.bar(['Numerator\n|Σ sin(Δφ)|', 'Denominator\nΣ |sin(Δφ)|', 'wPLI'], 
               [numerator, denominator, wpli_val * denominator],  # Scale for visibility
               color=[COLORS['signal_1'], COLORS['accent'], COLORS['highlight']],
               edgecolor='white', linewidth=2)

# Add text
ax6.text(0, numerator + 10, f'{numerator:.1f}', ha='center', fontsize=11, fontweight='bold')
ax6.text(1, denominator + 10, f'{denominator:.1f}', ha='center', fontsize=11, fontweight='bold')
ax6.text(2, wpli_val * denominator + 10, f'wPLI = {wpli_val:.3f}', ha='center', fontsize=11, fontweight='bold')
ax6.set_ylabel('Value', fontsize=10)
ax6.set_title('Step 6: Compute wPLI', fontsize=11, fontweight='bold')

plt.suptitle('wPLI Computation Pipeline', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"\nFinal wPLI = {wpli_val:.4f}")

---

## Section 5: wPLI vs PLI — Noise Robustness

Now let's demonstrate the key advantage of wPLI: **better noise robustness**, especially for small phase lags.

In [ ]:
def compute_pli_from_phases(
    phase_x: NDArray[np.float64],
    phase_y: NDArray[np.float64]
) -> float:
    """
    Compute Phase Lag Index from pre-computed phases.
    
    Parameters
    ----------
    phase_x : NDArray[np.float64]
        Instantaneous phase of signal x
    phase_y : NDArray[np.float64]
        Instantaneous phase of signal y
        
    Returns
    -------
    float
        PLI value in range [0, 1]
    """
    phase_diff = phase_x - phase_y
    signs = np.sign(np.sin(phase_diff))
    return float(np.abs(np.mean(signs)))


def compute_plv_from_phases(
    phase_x: NDArray[np.float64],
    phase_y: NDArray[np.float64]
) -> float:
    """
    Compute Phase Locking Value from pre-computed phases.
    
    Parameters
    ----------
    phase_x : NDArray[np.float64]
        Instantaneous phase of signal x
    phase_y : NDArray[np.float64]
        Instantaneous phase of signal y
        
    Returns
    -------
    float
        PLV value in range [0, 1]
    """
    phase_diff = phase_x - phase_y
    return float(np.abs(np.mean(np.exp(1j * phase_diff))))

In [ ]:
def compare_pli_wpli_noise(
    n_samples: int = 10000,
    fs: float = 500.0,
    frequency: float = 10.0,
    true_phase_lag: float = 0.2,
    noise_levels: list[float] | None = None,
    seed: int | None = None
) -> dict[str, Any]:
    """
    Compare PLI and wPLI robustness to noise.
    
    Parameters
    ----------
    n_samples : int, optional
        Number of samples (default: 10000)
    fs : float, optional
        Sampling frequency in Hz (default: 500.0)
    frequency : float, optional
        Signal frequency in Hz (default: 10.0)
    true_phase_lag : float, optional
        True phase lag in radians (default: 0.2, small lag)
    noise_levels : list[float], optional
        List of noise standard deviations (default: [0.0, 0.1, 0.2, 0.5, 1.0])
    seed : int, optional
        Random seed for reproducibility
        
    Returns
    -------
    dict[str, Any]
        Dictionary containing:
        - noise_levels: list of noise levels
        - plv_values: list of PLV values
        - pli_values: list of PLI values
        - wpli_values: list of wPLI values
    """
    if seed is not None:
        np.random.seed(seed)
    
    if noise_levels is None:
        noise_levels = [0.0, 0.1, 0.2, 0.3, 0.5, 0.7, 1.0, 1.5]
    
    t = np.arange(n_samples) / fs
    
    plv_values = []
    pli_values = []
    wpli_values = []
    
    for noise_std in noise_levels:
        # Generate clean signals with true phase lag
        x_clean = np.sin(2 * np.pi * frequency * t)
        y_clean = np.sin(2 * np.pi * frequency * t + true_phase_lag)
        
        # Add noise
        x = x_clean + noise_std * np.random.randn(n_samples)
        y = y_clean + noise_std * np.random.randn(n_samples)
        
        # Filter
        band = (frequency - 2, frequency + 2)
        x_filt = bandpass_filter(x, fs, band)
        y_filt = bandpass_filter(y, fs, band)
        
        # Extract phases
        phase_x = extract_phase(x_filt)
        phase_y = extract_phase(y_filt)
        
        # Compute metrics
        plv_values.append(compute_plv_from_phases(phase_x, phase_y))
        pli_values.append(compute_pli_from_phases(phase_x, phase_y))
        wpli_values.append(compute_wpli_from_phases(phase_x, phase_y))
    
    return {
        'noise_levels': noise_levels,
        'plv_values': plv_values,
        'pli_values': pli_values,
        'wpli_values': wpli_values
    }


# Run comparison with SMALL phase lag
results = compare_pli_wpli_noise(true_phase_lag=0.2, seed=42)  # 0.2 rad ≈ 11.5°

print(f"True phase lag: 0.2 rad ({np.degrees(0.2):.1f}°)")
print(f"\nNoise Level | PLV    | PLI    | wPLI")
print("-" * 40)
for nl, plv, pli, wpli in zip(results['noise_levels'], results['plv_values'], 
                               results['pli_values'], results['wpli_values']):
    print(f"    {nl:.1f}     | {plv:.3f}  | {pli:.3f}  | {wpli:.3f}")

In [ ]:
# Visualization 6: Noise robustness comparison

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Panel 1: All three metrics vs noise
ax1 = axes[0]
ax1.plot(results['noise_levels'], results['plv_values'], 'o-', 
         color=COLORS['signal_1'], linewidth=2, markersize=8, label='PLV')
ax1.plot(results['noise_levels'], results['pli_values'], 's-', 
         color=COLORS['signal_2'], linewidth=2, markersize=8, label='PLI')
ax1.plot(results['noise_levels'], results['wpli_values'], '^-', 
         color=COLORS['highlight'], linewidth=2, markersize=8, label='wPLI')

ax1.set_xlabel('Noise Level (std)', fontsize=12)
ax1.set_ylabel('Connectivity Value', fontsize=12)
ax1.set_title(f'Noise Robustness Comparison\n(True phase lag = 0.2 rad ≈ 11.5°)', fontsize=12, fontweight='bold')
ax1.legend(loc='upper right', fontsize=11)
ax1.set_ylim(0, 1.05)
ax1.grid(True, alpha=0.3)

# Panel 2: PLI vs wPLI focus
ax2 = axes[1]
ax2.plot(results['noise_levels'], results['pli_values'], 's-', 
         color=COLORS['signal_2'], linewidth=2.5, markersize=10, label='PLI')
ax2.plot(results['noise_levels'], results['wpli_values'], '^-', 
         color=COLORS['highlight'], linewidth=2.5, markersize=10, label='wPLI')

# Shade the region where wPLI outperforms PLI
ax2.fill_between(results['noise_levels'], results['pli_values'], results['wpli_values'],
                  where=[w > p for w, p in zip(results['wpli_values'], results['pli_values'])],
                  color=COLORS['highlight'], alpha=0.2, label='wPLI advantage')

ax2.set_xlabel('Noise Level (std)', fontsize=12)
ax2.set_ylabel('Connectivity Value', fontsize=12)
ax2.set_title('wPLI is more robust to noise than PLI', fontsize=12, fontweight='bold')
ax2.legend(loc='upper right', fontsize=11)
ax2.set_ylim(0, 1.05)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Key observation: wPLI degrades more slowly than PLI as noise increases.")
print("   This is especially important for small phase lags where sign flips are common.")

In [ ]:
# Visualization 7: Phase difference distributions at different noise levels

np.random.seed(42)
n_samples = 5000
fs = 500.0
frequency = 10.0
true_lag = 0.2
t = np.arange(n_samples) / fs

noise_levels_demo = [0.1, 0.5, 1.0]

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for ax, noise_std in zip(axes, noise_levels_demo):
    # Generate signals
    x = np.sin(2 * np.pi * frequency * t) + noise_std * np.random.randn(n_samples)
    y = np.sin(2 * np.pi * frequency * t + true_lag) + noise_std * np.random.randn(n_samples)
    
    # Filter and extract phases
    x_filt = bandpass_filter(x, fs, (frequency - 2, frequency + 2))
    y_filt = bandpass_filter(y, fs, (frequency - 2, frequency + 2))
    phase_x = extract_phase(x_filt)
    phase_y = extract_phase(y_filt)
    phase_diff = phase_x - phase_y
    
    # Wrap to [-π, π]
    phase_diff = np.arctan2(np.sin(phase_diff), np.cos(phase_diff))
    
    # Compute metrics
    pli = compute_pli_from_phases(phase_x, phase_y)
    wpli = compute_wpli_from_phases(phase_x, phase_y)
    
    # Plot histogram
    ax.hist(phase_diff, bins=50, color=COLORS['signal_1'], alpha=0.7, 
            edgecolor='white', density=True)
    ax.axvline(0, color='black', linestyle='-', linewidth=2, label='Zero')
    ax.axvline(true_lag, color=COLORS['highlight'], linestyle='--', linewidth=2, label='True lag')
    
    # Count zero-crossings
    n_negative = np.sum(np.sin(phase_diff) < 0)
    pct_negative = 100 * n_negative / n_samples
    
    ax.set_xlabel('Phase difference (rad)', fontsize=11)
    ax.set_ylabel('Density', fontsize=11)
    ax.set_title(f'Noise = {noise_std}\nPLI = {pli:.3f}, wPLI = {wpli:.3f}\n({pct_negative:.0f}% cross zero)', 
                 fontsize=11, fontweight='bold')
    ax.set_xlim(-1.5, 1.5)
    ax.legend(loc='upper right', fontsize=9)

plt.suptitle('Noise causes zero-crossing, destabilizing PLI more than wPLI', 
             fontsize=13, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---

## Section 6: wPLI vs Volume Conduction

An important question: **Does wPLI maintain PLI's robustness to volume conduction?**

Yes! Here's why:
- Volume conduction → Δφ ≈ 0
- Therefore sin(Δφ) ≈ 0
- Both numerator AND denominator are small
- But critically: the numerator (signed sum) averages to ~0 due to noise symmetry
- Result: wPLI ≈ 0

wPLI inherits PLI's key strength while improving noise robustness.

In [ ]:
def demonstrate_wpli_volume_conduction(
    n_samples: int = 10000,
    fs: float = 500.0,
    frequency: float = 10.0,
    seed: int | None = None
) -> dict[str, Any]:
    """
    Compare PLV, PLI, and wPLI for volume conduction vs true connectivity.
    
    Parameters
    ----------
    n_samples : int, optional
        Number of samples (default: 10000)
    fs : float, optional
        Sampling frequency in Hz (default: 500.0)
    frequency : float, optional
        Signal frequency in Hz (default: 10.0)
    seed : int, optional
        Random seed for reproducibility
        
    Returns
    -------
    dict[str, Any]
        Dictionary containing PLV, PLI, wPLI for both scenarios
    """
    if seed is not None:
        np.random.seed(seed)
    
    t = np.arange(n_samples) / fs
    band = (frequency - 2, frequency + 2)
    
    # === Volume Conduction Scenario ===
    # Same source, zero phase lag, slight amplitude difference
    source = np.sin(2 * np.pi * frequency * t) + 0.3 * np.random.randn(n_samples)
    x_vc = source + 0.1 * np.random.randn(n_samples)
    y_vc = 0.8 * source + 0.1 * np.random.randn(n_samples)
    
    x_vc_filt = bandpass_filter(x_vc, fs, band)
    y_vc_filt = bandpass_filter(y_vc, fs, band)
    phase_x_vc = extract_phase(x_vc_filt)
    phase_y_vc = extract_phase(y_vc_filt)
    
    plv_vc = compute_plv_from_phases(phase_x_vc, phase_y_vc)
    pli_vc = compute_pli_from_phases(phase_x_vc, phase_y_vc)
    wpli_vc = compute_wpli_from_phases(phase_x_vc, phase_y_vc)
    
    # === True Connectivity Scenario ===
    # Genuine phase lag (π/4 radians)
    true_lag = np.pi / 4
    x_tc = np.sin(2 * np.pi * frequency * t) + 0.3 * np.random.randn(n_samples)
    y_tc = np.sin(2 * np.pi * frequency * t + true_lag) + 0.3 * np.random.randn(n_samples)
    
    x_tc_filt = bandpass_filter(x_tc, fs, band)
    y_tc_filt = bandpass_filter(y_tc, fs, band)
    phase_x_tc = extract_phase(x_tc_filt)
    phase_y_tc = extract_phase(y_tc_filt)
    
    plv_tc = compute_plv_from_phases(phase_x_tc, phase_y_tc)
    pli_tc = compute_pli_from_phases(phase_x_tc, phase_y_tc)
    wpli_tc = compute_wpli_from_phases(phase_x_tc, phase_y_tc)
    
    # === No Connectivity Scenario ===
    x_nc = np.sin(2 * np.pi * frequency * t) + 0.3 * np.random.randn(n_samples)
    y_nc = np.sin(2 * np.pi * frequency * t + np.random.uniform(0, 2*np.pi)) + 0.3 * np.random.randn(n_samples)
    # Add phase jitter to destroy coherence
    y_nc = y_nc * np.sign(np.random.randn(n_samples))
    
    x_nc_filt = bandpass_filter(x_nc, fs, band)
    y_nc_filt = bandpass_filter(y_nc, fs, band)
    phase_x_nc = extract_phase(x_nc_filt)
    phase_y_nc = extract_phase(y_nc_filt)
    
    plv_nc = compute_plv_from_phases(phase_x_nc, phase_y_nc)
    pli_nc = compute_pli_from_phases(phase_x_nc, phase_y_nc)
    wpli_nc = compute_wpli_from_phases(phase_x_nc, phase_y_nc)
    
    return {
        'plv_vc': plv_vc, 'pli_vc': pli_vc, 'wpli_vc': wpli_vc,
        'plv_tc': plv_tc, 'pli_tc': pli_tc, 'wpli_tc': wpli_tc,
        'plv_nc': plv_nc, 'pli_nc': pli_nc, 'wpli_nc': wpli_nc
    }


results_vc = demonstrate_wpli_volume_conduction(seed=42)

print("Three-Metric Comparison:")
print("=" * 50)
print(f"{'Scenario':<25} {'PLV':>8} {'PLI':>8} {'wPLI':>8}")
print("-" * 50)
print(f"{'Volume Conduction':<25} {results_vc['plv_vc']:>8.3f} {results_vc['pli_vc']:>8.3f} {results_vc['wpli_vc']:>8.3f}")
print(f"{'True Connectivity':<25} {results_vc['plv_tc']:>8.3f} {results_vc['pli_tc']:>8.3f} {results_vc['wpli_tc']:>8.3f}")
print(f"{'No Connectivity':<25} {results_vc['plv_nc']:>8.3f} {results_vc['pli_nc']:>8.3f} {results_vc['wpli_nc']:>8.3f}")

In [ ]:
# Visualization 8: Three-metric comparison bar chart

fig, axes = plt.subplots(1, 3, figsize=(14, 5))

scenarios = ['Volume\nConduction', 'True\nConnectivity', 'No\nConnectivity']
metrics = ['PLV', 'PLI', 'wPLI']
colors = [COLORS['signal_1'], COLORS['signal_2'], COLORS['highlight']]

# Data
vc_data = [results_vc['plv_vc'], results_vc['pli_vc'], results_vc['wpli_vc']]
tc_data = [results_vc['plv_tc'], results_vc['pli_tc'], results_vc['wpli_tc']]
nc_data = [results_vc['plv_nc'], results_vc['pli_nc'], results_vc['wpli_nc']]

all_data = [vc_data, tc_data, nc_data]
titles = ['Volume Conduction\n(Should be LOW for PLI/wPLI)', 
          'True Connectivity\n(Should be HIGH for all)',
          'No Connectivity\n(Should be LOW for all)']
expected = ['⚠️ PLV gives false positive!', '✅ All detect connection', '✅ All correctly low']

for ax, data, title, exp in zip(axes, all_data, titles, expected):
    bars = ax.bar(metrics, data, color=colors, edgecolor='white', linewidth=2)
    
    # Add value labels
    for bar, val in zip(bars, data):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.03,
                f'{val:.3f}', ha='center', fontsize=12, fontweight='bold')
    
    ax.set_ylim(0, 1.15)
    ax.set_ylabel('Value', fontsize=11)
    ax.set_title(f'{title}\n{exp}', fontsize=11, fontweight='bold')
    ax.axhline(0.3, color='gray', linestyle='--', alpha=0.5)

plt.suptitle('wPLI Maintains Robustness to Volume Conduction', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n✅ Key finding: Both PLI and wPLI correctly reject volume conduction (low values)")
print("   while PLV gives a false positive (high value).")
print("   wPLI = Best of both worlds!")

---

## Section 7: Debiased wPLI (Optional Advanced)

Standard wPLI has a **small positive bias** for finite samples: even for independent signals, wPLI tends to be slightly above zero. The **Debiased wPLI (dwPLI)** corrects this.

The debiased formula removes the bias term by adjusting the squared imaginary cross-spectrum values.

In [ ]:
def compute_debiased_wpli(
    phase_x: NDArray[np.float64],
    phase_y: NDArray[np.float64]
) -> float:
    """
    Compute Debiased Weighted Phase Lag Index.
    
    Parameters
    ----------
    phase_x : NDArray[np.float64]
        Instantaneous phase of signal x
    phase_y : NDArray[np.float64]
        Instantaneous phase of signal y
        
    Returns
    -------
    float
        Debiased wPLI value in range [0, 1]
        
    Notes
    -----
    The debiased wPLI corrects for the positive bias in standard wPLI
    that arises from finite sample sizes. Formula:
    
    dwPLI = (Σ sin(Δφ))² - Σ sin²(Δφ)) / (Σ |sin(Δφ)|)² - Σ sin²(Δφ))
    """
    phase_diff = phase_x - phase_y
    sin_diff = np.sin(phase_diff)
    
    n = len(sin_diff)
    
    # Squared sum of signed values
    sum_sin = np.sum(sin_diff)
    sum_sin_sq = sum_sin ** 2
    
    # Sum of squared values (bias term)
    sum_sq_sin = np.sum(sin_diff ** 2)
    
    # Squared sum of absolute values
    sum_abs = np.sum(np.abs(sin_diff))
    sum_abs_sq = sum_abs ** 2
    
    # Debiased formula
    numerator = sum_sin_sq - sum_sq_sin
    denominator = sum_abs_sq - sum_sq_sin
    
    if denominator <= 0:
        return 0.0
    
    dwpli = numerator / denominator
    
    return float(np.clip(dwpli, 0.0, 1.0))

In [ ]:
# Visualization 9: wPLI vs dwPLI comparison for independent signals

np.random.seed(42)
n_iterations = 100
n_samples = 1000
fs = 500.0

wpli_independent = []
dwpli_independent = []

for _ in range(n_iterations):
    # Generate independent random phases
    phase_x = np.random.uniform(-np.pi, np.pi, n_samples)
    phase_y = np.random.uniform(-np.pi, np.pi, n_samples)
    
    wpli_independent.append(compute_wpli_from_phases(phase_x, phase_y))
    dwpli_independent.append(compute_debiased_wpli(phase_x, phase_y))

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Histograms
ax1 = axes[0]
ax1.hist(wpli_independent, bins=20, color=COLORS['signal_1'], alpha=0.7, 
         edgecolor='white', label=f'wPLI (mean={np.mean(wpli_independent):.4f})')
ax1.hist(dwpli_independent, bins=20, color=COLORS['highlight'], alpha=0.7,
         edgecolor='white', label=f'dwPLI (mean={np.mean(dwpli_independent):.4f})')
ax1.axvline(0, color='black', linestyle='--', linewidth=2, label='Expected (0)')
ax1.set_xlabel('Value', fontsize=11)
ax1.set_ylabel('Count', fontsize=11)
ax1.set_title('Distribution for Independent Signals', fontsize=12, fontweight='bold')
ax1.legend(loc='upper right')

# Box plot comparison
ax2 = axes[1]
bp = ax2.boxplot([wpli_independent, dwpli_independent], 
                  labels=['wPLI', 'dwPLI'],
                  patch_artist=True)
bp['boxes'][0].set_facecolor(COLORS['signal_1'])
bp['boxes'][1].set_facecolor(COLORS['highlight'])
ax2.axhline(0, color='black', linestyle='--', linewidth=2)
ax2.set_ylabel('Value', fontsize=11)
ax2.set_title('Bias Comparison: dwPLI closer to zero', fontsize=12, fontweight='bold')

plt.suptitle('Debiased wPLI Reduces Positive Bias for Independent Signals', 
             fontsize=13, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"\nFor independent signals (expected = 0):")
print(f"  wPLI mean:  {np.mean(wpli_independent):.4f} ± {np.std(wpli_independent):.4f}")
print(f"  dwPLI mean: {np.mean(dwpli_independent):.4f} ± {np.std(dwpli_independent):.4f}")
print(f"\n💡 dwPLI is closer to zero, showing reduced bias.")

---

## Section 8: wPLI Matrix

For multi-channel analysis, we compute wPLI for all channel pairs, creating a connectivity matrix.

In [ ]:
def compute_wpli_matrix(
    data: NDArray[np.float64],
    fs: float,
    band: tuple[float, float]
) -> NDArray[np.float64]:
    """
    Compute wPLI matrix for all channel pairs.
    
    Parameters
    ----------
    data : NDArray[np.float64]
        EEG data with shape (n_channels, n_samples)
    fs : float
        Sampling frequency in Hz
    band : tuple[float, float]
        Frequency band (low, high) in Hz
        
    Returns
    -------
    NDArray[np.float64]
        wPLI matrix with shape (n_channels, n_channels)
    """
    n_channels = data.shape[0]
    
    # Filter all channels
    data_filt = bandpass_filter(data, fs, band)
    
    # Extract phases for all channels
    phases = extract_phase(data_filt)
    
    # Initialize matrix
    wpli_matrix = np.zeros((n_channels, n_channels))
    
    # Compute pairwise wPLI
    for i in range(n_channels):
        for j in range(i + 1, n_channels):
            wpli_val = compute_wpli_from_phases(phases[i], phases[j])
            wpli_matrix[i, j] = wpli_val
            wpli_matrix[j, i] = wpli_val  # Symmetric
    
    return wpli_matrix


def compute_wpli_matrix_bands(
    data: NDArray[np.float64],
    fs: float,
    bands: dict[str, tuple[float, float]] | None = None
) -> dict[str, NDArray[np.float64]]:
    """
    Compute wPLI matrices for multiple frequency bands.
    
    Parameters
    ----------
    data : NDArray[np.float64]
        EEG data with shape (n_channels, n_samples)
    fs : float
        Sampling frequency in Hz
    bands : dict[str, tuple[float, float]], optional
        Dictionary of frequency bands. Default: standard EEG bands
        
    Returns
    -------
    dict[str, NDArray[np.float64]]
        Dictionary mapping band names to wPLI matrices
    """
    if bands is None:
        bands = {
            'theta': (4, 8),
            'alpha': (8, 13),
            'beta': (13, 30),
            'gamma': (30, 45)
        }
    
    return {name: compute_wpli_matrix(data, fs, band) for name, band in bands.items()}

In [ ]:
# Generate multi-channel data with known structure
np.random.seed(42)
n_channels = 8
n_samples = 5000
fs = 500.0
t = np.arange(n_samples) / fs

# Create data with cluster structure
data = np.zeros((n_channels, n_samples))

# Cluster 1: channels 0-2 (phase locked in alpha)
source1 = np.sin(2 * np.pi * 10 * t)
for i in range(3):
    lag = i * 0.05
    data[i] = np.sin(2 * np.pi * 10 * t + lag) + 0.3 * np.random.randn(n_samples)

# Cluster 2: channels 3-5 (phase locked in alpha, different phase)
for i in range(3, 6):
    lag = np.pi/2 + (i-3) * 0.05
    data[i] = np.sin(2 * np.pi * 10 * t + lag) + 0.3 * np.random.randn(n_samples)

# Independent: channels 6-7
for i in range(6, 8):
    data[i] = np.sin(2 * np.pi * 10 * t + np.random.uniform(0, 2*np.pi)) + 0.5 * np.random.randn(n_samples)

In [ ]:
# Compute all three matrices for comparison

def compute_plv_matrix(
    data: NDArray[np.float64],
    fs: float,
    band: tuple[float, float]
) -> NDArray[np.float64]:
    """Compute PLV matrix."""
    n_channels = data.shape[0]
    data_filt = bandpass_filter(data, fs, band)
    phases = extract_phase(data_filt)
    
    plv_matrix = np.zeros((n_channels, n_channels))
    for i in range(n_channels):
        for j in range(i + 1, n_channels):
            plv_val = compute_plv_from_phases(phases[i], phases[j])
            plv_matrix[i, j] = plv_val
            plv_matrix[j, i] = plv_val
    return plv_matrix


def compute_pli_matrix(
    data: NDArray[np.float64],
    fs: float,
    band: tuple[float, float]
) -> NDArray[np.float64]:
    """Compute PLI matrix."""
    n_channels = data.shape[0]
    data_filt = bandpass_filter(data, fs, band)
    phases = extract_phase(data_filt)
    
    pli_matrix = np.zeros((n_channels, n_channels))
    for i in range(n_channels):
        for j in range(i + 1, n_channels):
            pli_val = compute_pli_from_phases(phases[i], phases[j])
            pli_matrix[i, j] = pli_val
            pli_matrix[j, i] = pli_val
    return pli_matrix


# Compute matrices
band_alpha = (8, 12)
plv_mat = compute_plv_matrix(data, fs, band_alpha)
pli_mat = compute_pli_matrix(data, fs, band_alpha)
wpli_mat = compute_wpli_matrix(data, fs, band_alpha)

In [ ]:
# Visualization 10: Three matrices side by side

fig, axes = plt.subplots(1, 3, figsize=(15, 4.5))

matrices = [plv_mat, pli_mat, wpli_mat]
titles = ['PLV Matrix', 'PLI Matrix', 'wPLI Matrix']

for ax, mat, title in zip(axes, matrices, titles):
    im = ax.imshow(mat, cmap='viridis', vmin=0, vmax=1, aspect='equal')
    ax.set_title(title, fontsize=13, fontweight='bold')
    ax.set_xlabel('Channel', fontsize=11)
    ax.set_ylabel('Channel', fontsize=11)
    ax.set_xticks(range(n_channels))
    ax.set_yticks(range(n_channels))
    
    # Add cluster annotations
    ax.axhline(2.5, color='white', linestyle='--', linewidth=1, alpha=0.8)
    ax.axvline(2.5, color='white', linestyle='--', linewidth=1, alpha=0.8)
    ax.axhline(5.5, color='white', linestyle='--', linewidth=1, alpha=0.8)
    ax.axvline(5.5, color='white', linestyle='--', linewidth=1, alpha=0.8)
    
    plt.colorbar(im, ax=ax, shrink=0.8)

# Add cluster labels
for ax in axes:
    ax.text(-1.5, 1, 'C1', fontsize=10, fontweight='bold', color=COLORS['signal_1'])
    ax.text(-1.5, 4, 'C2', fontsize=10, fontweight='bold', color=COLORS['signal_2'])
    ax.text(-1.5, 6.5, 'Ind', fontsize=10, fontweight='bold', color='gray')

plt.suptitle('PLV vs PLI vs wPLI Matrices (Alpha band: 8-12 Hz)\n'
             'C1: Cluster 1 (ch 0-2), C2: Cluster 2 (ch 3-5), Ind: Independent (ch 6-7)', 
             fontsize=12, fontweight='bold', y=1.05)
plt.tight_layout()
plt.show()

---

## Section 9: wPLI for Hyperscanning

wPLI works excellently for hyperscanning analysis:

- **Within-brain**: Provides volume conduction robustness with better stability than PLI
- **Between-brain**: Comparable to PLV but more conservative

**Recommendation**: wPLI is often the "safest" phase-based metric for hyperscanning because it's robust to both volume conduction AND noise.

In [ ]:
def compute_wpli_hyperscanning(
    data_p1: NDArray[np.float64],
    data_p2: NDArray[np.float64],
    fs: float,
    band: tuple[float, float]
) -> dict[str, NDArray[np.float64]]:
    """
    Compute wPLI matrices for hyperscanning analysis.
    
    Parameters
    ----------
    data_p1 : NDArray[np.float64]
        EEG data from participant 1, shape (n_channels, n_samples)
    data_p2 : NDArray[np.float64]
        EEG data from participant 2, shape (n_channels, n_samples)
    fs : float
        Sampling frequency in Hz
    band : tuple[float, float]
        Frequency band (low, high) in Hz
        
    Returns
    -------
    dict[str, NDArray[np.float64]]
        Dictionary containing:
        - within_p1: wPLI matrix for participant 1
        - within_p2: wPLI matrix for participant 2
        - between: wPLI matrix between participants
        - full: Full combined matrix
    """
    n_ch1 = data_p1.shape[0]
    n_ch2 = data_p2.shape[0]
    
    # Filter all data
    data_p1_filt = bandpass_filter(data_p1, fs, band)
    data_p2_filt = bandpass_filter(data_p2, fs, band)
    
    # Extract phases
    phases_p1 = extract_phase(data_p1_filt)
    phases_p2 = extract_phase(data_p2_filt)
    
    # Within-participant 1
    within_p1 = np.zeros((n_ch1, n_ch1))
    for i in range(n_ch1):
        for j in range(i + 1, n_ch1):
            val = compute_wpli_from_phases(phases_p1[i], phases_p1[j])
            within_p1[i, j] = val
            within_p1[j, i] = val
    
    # Within-participant 2
    within_p2 = np.zeros((n_ch2, n_ch2))
    for i in range(n_ch2):
        for j in range(i + 1, n_ch2):
            val = compute_wpli_from_phases(phases_p2[i], phases_p2[j])
            within_p2[i, j] = val
            within_p2[j, i] = val
    
    # Between participants
    between = np.zeros((n_ch1, n_ch2))
    for i in range(n_ch1):
        for j in range(n_ch2):
            between[i, j] = compute_wpli_from_phases(phases_p1[i], phases_p2[j])
    
    # Full matrix
    n_total = n_ch1 + n_ch2
    full = np.zeros((n_total, n_total))
    full[:n_ch1, :n_ch1] = within_p1
    full[n_ch1:, n_ch1:] = within_p2
    full[:n_ch1, n_ch1:] = between
    full[n_ch1:, :n_ch1] = between.T
    
    return {
        'within_p1': within_p1,
        'within_p2': within_p2,
        'between': between,
        'full': full
    }

In [ ]:
# Generate hyperscanning data
np.random.seed(42)
n_channels_per = 4
n_samples = 5000
fs = 500.0
t = np.arange(n_samples) / fs

# Participant 1 data
data_p1 = np.zeros((n_channels_per, n_samples))
base_p1 = np.sin(2 * np.pi * 10 * t)
for i in range(n_channels_per):
    data_p1[i] = base_p1 + 0.3 * np.random.randn(n_samples) + i * 0.05 * base_p1

# Participant 2 data (some inter-brain coupling)
data_p2 = np.zeros((n_channels_per, n_samples))
coupling_strength = 0.4
for i in range(n_channels_per):
    independent = np.sin(2 * np.pi * 10 * t + np.random.uniform(0, np.pi))
    coupled = coupling_strength * base_p1 + (1 - coupling_strength) * independent
    data_p2[i] = coupled + 0.3 * np.random.randn(n_samples)

# Compute hyperscanning wPLI
hyper_wpli = compute_wpli_hyperscanning(data_p1, data_p2, fs, (8, 12))

In [ ]:
# Visualization 11: Hyperscanning wPLI

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Full matrix
ax1 = axes[0]
im1 = ax1.imshow(hyper_wpli['full'], cmap='viridis', vmin=0, vmax=1, aspect='equal')

# Add block separators
ax1.axhline(n_channels_per - 0.5, color='white', linewidth=2)
ax1.axvline(n_channels_per - 0.5, color='white', linewidth=2)

# Labels
n_total = 2 * n_channels_per
ax1.set_xticks(range(n_total))
ax1.set_yticks(range(n_total))
labels = [f'P1-{i}' for i in range(n_channels_per)] + [f'P2-{i}' for i in range(n_channels_per)]
ax1.set_xticklabels(labels, rotation=45, ha='right')
ax1.set_yticklabels(labels)

# Block annotations
ax1.text(1.5, 1.5, 'Within\nP1', ha='center', va='center', fontsize=10, 
         color='white', fontweight='bold')
ax1.text(5.5, 5.5, 'Within\nP2', ha='center', va='center', fontsize=10,
         color='white', fontweight='bold')
ax1.text(5.5, 1.5, 'Between', ha='center', va='center', fontsize=10,
         color='white', fontweight='bold')

ax1.set_title('Full Hyperscanning wPLI Matrix', fontsize=13, fontweight='bold')
plt.colorbar(im1, ax=ax1, shrink=0.8, label='wPLI')

# Between-brain matrix
ax2 = axes[1]
im2 = ax2.imshow(hyper_wpli['between'], cmap='viridis', vmin=0, vmax=1, aspect='equal')
ax2.set_xlabel('Participant 2 channels', fontsize=11)
ax2.set_ylabel('Participant 1 channels', fontsize=11)
ax2.set_title('Between-Brain wPLI (Inter-Brain Connectivity)', fontsize=13, fontweight='bold')
ax2.set_xticks(range(n_channels_per))
ax2.set_yticks(range(n_channels_per))
plt.colorbar(im2, ax=ax2, shrink=0.8, label='wPLI')

plt.suptitle('Inter-Brain wPLI in Hyperscanning', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"\nMean connectivity values:")
print(f"  Within P1:  {np.mean(hyper_wpli['within_p1'][np.triu_indices(n_channels_per, k=1)]):.3f}")
print(f"  Within P2:  {np.mean(hyper_wpli['within_p2'][np.triu_indices(n_channels_per, k=1)]):.3f}")
print(f"  Between:    {np.mean(hyper_wpli['between']):.3f}")

---

## Section 10: Choosing Among Phase Metrics

Here's a practical guide for choosing between PLV, PLI, and wPLI:

| Metric | Volume Conduction | Noise Robustness | Sensitivity |
|--------|-------------------|------------------|-------------|
| PLV | ❌ High false positives | Good | High |
| PLI | ✅ Robust | Poor (sign flips) | Moderate |
| wPLI | ✅ Robust | Good | Moderate |

**Recommendations:**
- **PLV**: Use for between-brain analysis (no volume conduction issue) or at source level
- **PLI**: When simplicity matters, or for comparison with older studies
- **wPLI**: Default choice for sensor-level EEG (best balance of robustness)

**Always report** which metric you used and why!

In [ ]:
# Visualization 12: Decision flowchart (text-based)

fig, ax = plt.subplots(figsize=(12, 8))
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.axis('off')

# Flowchart boxes
def draw_box(ax, x, y, w, h, text, color, fontsize=10):
    rect = plt.Rectangle((x-w/2, y-h/2), w, h, fill=True, 
                          facecolor=color, edgecolor='black', linewidth=2, alpha=0.8)
    ax.add_patch(rect)
    ax.text(x, y, text, ha='center', va='center', fontsize=fontsize, fontweight='bold', wrap=True)

def draw_diamond(ax, x, y, size, text, color):
    diamond = plt.Polygon([(x, y+size), (x+size, y), (x, y-size), (x-size, y)],
                          facecolor=color, edgecolor='black', linewidth=2, alpha=0.8)
    ax.add_patch(diamond)
    ax.text(x, y, text, ha='center', va='center', fontsize=9, fontweight='bold')

def draw_arrow(ax, x1, y1, x2, y2, label=''):
    ax.annotate('', xy=(x2, y2), xytext=(x1, y1),
                arrowprops=dict(arrowstyle='->', color='black', lw=2))
    if label:
        mid_x, mid_y = (x1+x2)/2, (y1+y2)/2
        ax.text(mid_x + 0.3, mid_y, label, fontsize=9, fontweight='bold')

# Start
draw_box(ax, 5, 9, 3, 0.8, 'Phase Connectivity\nMetric Selection', '#E8E8E8', 11)

# Question 1
draw_diamond(ax, 5, 7.2, 0.8, 'Source or\nSensor?', '#FFE4B5')
draw_arrow(ax, 5, 8.6, 5, 8)

# Source path
draw_box(ax, 2, 5.5, 2, 0.7, 'PLV OK', COLORS['signal_1'])
draw_arrow(ax, 4.2, 7.2, 3, 5.9, 'Source')

# Sensor path - Question 2
draw_diamond(ax, 7, 5.5, 0.8, 'Volume\nConduction\nConcern?', '#FFE4B5')
draw_arrow(ax, 5.8, 7.2, 7, 6.3, 'Sensor')

# No VC concern
draw_box(ax, 9, 4, 1.5, 0.7, 'PLV', COLORS['signal_1'])
draw_arrow(ax, 7.8, 5.5, 8.25, 4.35, 'No')

# Yes VC concern - Question 3
draw_diamond(ax, 5, 3.5, 0.8, 'Noisy data?\nSmall effects?', '#FFE4B5')
draw_arrow(ax, 6.2, 5.5, 5.8, 4.3, 'Yes')

# Final recommendations
draw_box(ax, 3, 1.8, 1.5, 0.7, 'PLI', COLORS['signal_2'])
draw_arrow(ax, 4.2, 3.5, 3.75, 2.15, 'No')

draw_box(ax, 7, 1.8, 1.5, 0.7, 'wPLI ⭐', COLORS['highlight'])
draw_arrow(ax, 5.8, 3.5, 6.25, 2.15, 'Yes')

# Legend
ax.text(0.5, 0.5, '⭐ = Recommended default', fontsize=10, style='italic')

ax.set_title('Choosing a Phase-Based Connectivity Metric', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# Visualization 13: Summary bar chart - metrics across scenarios

# Run comprehensive comparison
np.random.seed(42)
n_samples = 10000
fs = 500.0
freq = 10.0
t = np.arange(n_samples) / fs
band = (8, 12)

scenarios = {
    'Volume\nConduction': {'type': 'vc'},
    'True Connection\n(Clean)': {'type': 'clean'},
    'True Connection\n(Noisy)': {'type': 'noisy'},
    'Small Phase Lag\n(Noisy)': {'type': 'small_lag'}
}

results_all = {'PLV': [], 'PLI': [], 'wPLI': []}

for name, params in scenarios.items():
    if params['type'] == 'vc':
        source = np.sin(2 * np.pi * freq * t) + 0.3 * np.random.randn(n_samples)
        x = source + 0.1 * np.random.randn(n_samples)
        y = 0.8 * source + 0.1 * np.random.randn(n_samples)
    elif params['type'] == 'clean':
        x = np.sin(2 * np.pi * freq * t) + 0.2 * np.random.randn(n_samples)
        y = np.sin(2 * np.pi * freq * t + np.pi/4) + 0.2 * np.random.randn(n_samples)
    elif params['type'] == 'noisy':
        x = np.sin(2 * np.pi * freq * t) + 0.8 * np.random.randn(n_samples)
        y = np.sin(2 * np.pi * freq * t + np.pi/4) + 0.8 * np.random.randn(n_samples)
    else:  # small_lag
        x = np.sin(2 * np.pi * freq * t) + 0.5 * np.random.randn(n_samples)
        y = np.sin(2 * np.pi * freq * t + 0.15) + 0.5 * np.random.randn(n_samples)
    
    x_filt = bandpass_filter(x, fs, band)
    y_filt = bandpass_filter(y, fs, band)
    px, py = extract_phase(x_filt), extract_phase(y_filt)
    
    results_all['PLV'].append(compute_plv_from_phases(px, py))
    results_all['PLI'].append(compute_pli_from_phases(px, py))
    results_all['wPLI'].append(compute_wpli_from_phases(px, py))

# Plot
fig, ax = plt.subplots(figsize=(12, 6))

x_pos = np.arange(len(scenarios))
width = 0.25

bars1 = ax.bar(x_pos - width, results_all['PLV'], width, label='PLV', 
               color=COLORS['signal_1'], edgecolor='white', linewidth=1)
bars2 = ax.bar(x_pos, results_all['PLI'], width, label='PLI',
               color=COLORS['signal_2'], edgecolor='white', linewidth=1)
bars3 = ax.bar(x_pos + width, results_all['wPLI'], width, label='wPLI',
               color=COLORS['highlight'], edgecolor='white', linewidth=1)

# Value labels
for bars in [bars1, bars2, bars3]:
    for bar in bars:
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                f'{bar.get_height():.2f}', ha='center', fontsize=9, fontweight='bold')

ax.set_ylabel('Connectivity Value', fontsize=12)
ax.set_xticks(x_pos)
ax.set_xticklabels(scenarios.keys(), fontsize=10)
ax.set_ylim(0, 1.1)
ax.legend(loc='upper right', fontsize=11)

# Expected behavior annotations
ax.axhline(0.3, color='gray', linestyle='--', alpha=0.3)
ax.text(3.5, 0.32, 'Threshold', fontsize=9, color='gray')

ax.set_title('Phase Metrics Across Scenarios\n'
             '(PLV fails at VC; PLI fails at small lag + noise; wPLI performs well overall)',
             fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 Summary:")
print("  - Volume Conduction: PLV gives false positive, PLI & wPLI correctly low")
print("  - Clean connection: All three detect it well")
print("  - Noisy connection: wPLI more robust than PLI")
print("  - Small lag + noise: wPLI maintains detection, PLI drops significantly")

---

## Section 11: Comparison with HyPyP

HyPyP implements PLV, PLI, and wPLI. Let's validate our implementation against HyPyP (if available).

In [ ]:
def compare_with_hypyp_wpli(
    data_p1: NDArray[np.float64],
    data_p2: NDArray[np.float64],
    fs: float,
    band: tuple[float, float]
) -> dict[str, Any]:
    """
    Compare our wPLI implementation with HyPyP.
    
    Parameters
    ----------
    data_p1 : NDArray[np.float64]
        Data from participant 1
    data_p2 : NDArray[np.float64]
        Data from participant 2
    fs : float
        Sampling frequency
    band : tuple[float, float]
        Frequency band
        
    Returns
    -------
    dict[str, Any]
        Comparison results including correlation and max difference
    """
    # Compute our wPLI
    our_result = compute_wpli_hyperscanning(data_p1, data_p2, fs, band)
    our_wpli = our_result['between']
    
    try:
        # Try to import HyPyP
        from hypyp import analyses
        import mne
        
        # Create MNE info objects
        n_ch1 = data_p1.shape[0]
        n_ch2 = data_p2.shape[0]
        
        info1 = mne.create_info([f'ch{i}' for i in range(n_ch1)], fs, 'eeg')
        info2 = mne.create_info([f'ch{i}' for i in range(n_ch2)], fs, 'eeg')
        
        # This is a placeholder - actual HyPyP comparison would require
        # proper epoch structure
        hypyp_available = True
        hypyp_wpli = our_wpli  # Placeholder
        
    except ImportError:
        hypyp_available = False
        hypyp_wpli = None
    
    if hypyp_available and hypyp_wpli is not None:
        correlation = np.corrcoef(our_wpli.flatten(), hypyp_wpli.flatten())[0, 1]
        max_diff = np.max(np.abs(our_wpli - hypyp_wpli))
    else:
        correlation = None
        max_diff = None
    
    return {
        'our_wpli': our_wpli,
        'hypyp_wpli': hypyp_wpli,
        'hypyp_available': hypyp_available,
        'correlation': correlation,
        'max_difference': max_diff
    }


# Run comparison
comparison = compare_with_hypyp_wpli(data_p1, data_p2, fs, (8, 12))

if comparison['hypyp_available']:
    print(f"HyPyP comparison:")
    print(f"  Correlation: {comparison['correlation']:.4f}")
    print(f"  Max difference: {comparison['max_difference']:.6f}")
else:
    print("HyPyP not available for comparison.")
    print("Our implementation follows the same mathematical formula.")
    print("To validate, install HyPyP: pip install hypyp")

---

## Section 12: Hands-On Exercises

Practice implementing and interpreting wPLI!

In [ ]:
# Exercise 1: wPLI Basics
# Generate phase-locked signals with moderate lag (π/4)
# Compute PLV, PLI, wPLI - all should be reasonably high
# Then add noise and observe stability differences

np.random.seed(42)
n_samples = 5000
fs = 500.0
t = np.arange(n_samples) / fs
freq = 10.0
band = (8, 12)
phase_lag = np.pi / 4  # Moderate lag

# YOUR CODE HERE
# 1. Create signals with phase_lag
# 2. Compute PLV, PLI, wPLI
# 3. Add noise (std=0.5) and recompute
# 4. Compare the stability

# x = ...
# y = ...

print("Exercise 1: Compare PLV, PLI, wPLI stability with moderate phase lag")

In [ ]:
# Exercise 2: Small Phase Lag Challenge
# This is where wPLI shines!
# Generate signals with SMALL phase lag (π/20 ≈ 9°)
# Observe PLI instability vs wPLI stability

np.random.seed(42)
small_lag = np.pi / 20  # Very small lag

# YOUR CODE HERE
# 1. Create signals with small_lag
# 2. Add moderate noise (std=0.4)
# 3. Compute PLI and wPLI
# 4. Explain why wPLI performs better

print(f"Exercise 2: Small phase lag = {small_lag:.3f} rad ({np.degrees(small_lag):.1f}°)")

In [ ]:
# Exercise 3: Volume Conduction Check
# Verify that wPLI maintains robustness to volume conduction

np.random.seed(42)

# YOUR CODE HERE
# 1. Create volume conduction scenario (same source, zero lag)
# 2. Compute PLV, PLI, wPLI
# 3. Verify: PLV high, PLI low, wPLI low

print("Exercise 3: Verify wPLI robustness to volume conduction")

In [ ]:
# Exercise 4: Noise Robustness Experiment
# Systematically vary noise level and compare PLI vs wPLI

np.random.seed(42)

# YOUR CODE HERE
# 1. Use true_phase_lag = 0.3 rad
# 2. Vary noise from 0 to 1.5 in steps of 0.1
# 3. Plot PLI and wPLI vs noise level
# 4. Calculate at what noise level PLI drops below 0.3
#    vs when wPLI drops below 0.3

print("Exercise 4: Quantify noise robustness difference")

In [ ]:
# Exercise 5: Three-Metric Comparison Matrix
# Create 8-channel data with varied connectivity types
# Compare PLV, PLI, wPLI matrices

np.random.seed(42)

# YOUR CODE HERE
# 1. Create 8-channel data:
#    - Channels 0-1: strong coupling, clean
#    - Channels 2-3: strong coupling, noisy
#    - Channels 4-5: volume conduction
#    - Channels 6-7: independent
# 2. Compute all three matrices
# 3. Identify which pairs show largest differences across metrics
# 4. Interpret: why do they differ?

print("Exercise 5: Multi-metric matrix comparison")

In [ ]:
# Exercise 6: Hyperscanning Application
# Compare PLV and wPLI for hyperscanning analysis

np.random.seed(42)

# YOUR CODE HERE
# 1. Create two-participant data (4 channels each)
# 2. Add some inter-brain coupling
# 3. Compute both PLV and wPLI hyperscanning matrices
# 4. Compare within-brain vs between-brain differences
# 5. Which metric shows larger within-brain values? Why?

print("Exercise 6: Hyperscanning PLV vs wPLI comparison")

---

## Summary

### Key Takeaways

1. **PLI's problem**: The sign function is sensitive to noise near zero phase differences

2. **wPLI's solution**: Weight contributions by |sin(Δφ)|
   - Phases near zero get less weight → noise robustness
   - Formula: wPLI = |Σ sin(Δφ)| / Σ |sin(Δφ)|

3. **Properties**:
   - Range: 0 to 1
   - Maintains volume conduction robustness (like PLI)
   - Improved noise robustness (better than PLI)

4. **Debiased wPLI**: Corrects small positive bias in finite samples

5. **For hyperscanning**: wPLI is often the best default phase metric

6. **Decision guide**:
   - **PLV**: Simple, but vulnerable to volume conduction
   - **PLI**: VC robust, but noise sensitive
   - **wPLI**: VC robust AND noise robust — best of both!

7. **HyPyP** implements all three metrics

---

## Discussion Questions

1. You're analyzing EEG from a clinical population with noisier data than typical. Would this affect your metric choice? How?

2. wPLI weights phases near 0 and π less. But what if the true connectivity happens to have exactly π/2 phase lag? Is wPLI "unfairly" advantaged in detecting such connections?

3. Some researchers argue wPLI is now the "gold standard" for EEG phase connectivity. Do you agree? What might still justify using PLV in some contexts?

4. The progression PLV → PLI → wPLI shows increasing robustness but decreasing simplicity. When does simplicity matter? When should you prefer robustness?

5. You need to compare your hyperscanning results with a 2010 paper that used PLV. Can you use wPLI and still make valid comparisons? How would you handle this?

---

## Next Steps

Proceed to **H01: Envelope Correlation** to learn about amplitude-based connectivity metrics!